In [ ]:
# Install Pathway and Bokeh
!pip install pathway bokeh --quiet

# Imports libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

# Load and preprocess data
df = pd.read_csv('/content/dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)


# Save stream-ready CSV
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)

# Define the schema for streaming data
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int


# Simulate a real-time data stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Convert timestamp and derive day-level information
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Model 1: Daily Dynamic Pricing with Aggregation
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(duration=timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        occ_min = pw.reducers.min(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# Visualization with Bokeh + Panel
pn.extension()


# Define visualization logic
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Model 1 - Daily Price Based on Demand Fluctuation",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", size=6, color="red", source=source)
    return fig

# Generate and serve visualization
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the real-time data stream processing
pw.run()

Output()

In [ ]:
#Install Pathway and Bokeh
!pip install pathway bokeh --quiet

# Imports libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

# Load and preprocess data
df = pd.read_csv('/content/dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)


# Save stream-ready CSV
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'IsSpecialDay']].to_csv("parking_stream_model2.csv", index=False)

# Define schema for Model 2
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    IsSpecialDay: int

# Simulate stream
data = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Model 2: Demand-Sensitive Pricing with Contextual Features
BASE_PRICE = 10.0
ALPHA = 3.0  # weight for occupancy
BETA = 0.5   # weight for queue length
GAMMA = 2.0  # special day multiplier

# Compute price directly using pw.apply

def compute_price(occupancy, capacity, queue_len, special_day):
    demand_score = (
        ALPHA * (occupancy / capacity) +
        BETA * (queue_len / 10.0)
    ) * (1 + (special_day * GAMMA))
    return round(BASE_PRICE * (1 + demand_score), 2)

data_enriched = data_with_time.with_columns(
    price = pw.apply(
        compute_price,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.IsSpecialDay
    )
)

# Visualization with Bokeh + Panel
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Model 2 - Demand-Sensitive Dynamic Pricing",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.circle("t", "price", size=6, color="orange", source=source)
    return fig

# Plot and serve
viz = data_enriched.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the real-time data stream processing
pw.run()

Output()

In [ ]:
# Install Pathway and Bokeh
!pip install pathway bokeh --quiet

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

# Load and preprocess data
df = pd.read_csv('/content/dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map categorical traffic values to float levels
traffic_map = {'low': 0.2, 'medium': 0.5, 'high': 0.8, 'very high': 1.0}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.0)

# Save filtered columns for Model 3
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'IsSpecialDay', 'TrafficConditionNearby']].to_csv("parking_stream_model3.csv", index=False)

# Define schema for Model 3
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    TrafficConditionNearby: float

# Simulate stream
data = pw.demo.replay_csv("parking_stream_model3.csv", schema=ParkingSchema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Model 3: Enhanced Pricing with Traffic Conditions
BASE_PRICE = 10.0
ALPHA = 2.0  # weight for occupancy
BETA = 0.4   # weight for queue length
GAMMA = 1.5  # special day multiplier
DELTA = 2.0  # traffic condition multiplier

# Define pricing function
def compute_price(occupancy, capacity, queue_len, special_day, traffic):
    try:
        demand_score = (
            ALPHA * (occupancy / capacity) +
            BETA * (queue_len / 10.0)
        ) * (1 + special_day * GAMMA)
        traffic_boost = 1 + (traffic * DELTA)
        return round(BASE_PRICE * (1 + demand_score) * traffic_boost, 2)
    except Exception:
        return BASE_PRICE

# Apply enhanced pricing logic
data_enriched = data_with_time.with_columns(
    price = pw.apply(
        compute_price,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.IsSpecialDay,
        pw.this.TrafficConditionNearby
    )
)

# Visualization with Bokeh + Panel
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Model 3 - Traffic-Aware Pricing",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="purple")
    fig.scatter(x="t", y="price", size=6, color="gold", source=source)
    return fig

# Plot and serve
viz = data_enriched.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the real-time data stream processing
pw.run()

Output()